In [2]:
# import modules
import xml.etree.ElementTree as ET
import pandas as pd
import sys
import string
import re
import nltk
import sys
import os
import gensim

# Add the src directory to sys.path
sys.path.append(os.path.abspath(r'C:\dev\NLP2RE_Sandbox\src'))


print(sys.version)
print(sys.executable)

3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]
c:\ProgramData\anaconda3\python.exe


In [3]:
# Specify the path to your XML file and namespace
xml_file_path = r'C:\dev\NLP2RE_Sandbox\data\work_data\2007-ertms.xml'
namespace = {'ns': 'req_document.xsd'}

In [5]:
# Parse .xml to df
from utils import parse_xml

# import utils.ParseXML as ParseXML
df = parse_xml.process_xml_with_namespace(xml_file_path, namespace)
df.tail(10)

,tag,text,id,path
621,meaning,Transmission of ETCS information from a train ...,,req_document/p/glossary/glossary_item/meaning
622,term,Train trip,,req_document/p/glossary/glossary_item/term
623,meaning,"Is used when a train passes a ""danger"" signal,...",,req_document/p/glossary/glossary_item/meaning
624,term,Warning,,req_document/p/glossary/glossary_item/term
625,meaning,Audible and/or visual indication to alert the ...,,req_document/p/glossary/glossary_item/meaning
626,term,Wheelslip,,req_document/p/glossary/glossary_item/term
627,meaning,When a traction-driven wheel loses adhesion wi...,,req_document/p/glossary/glossary_item/meaning
628,term,Wheelslide,,req_document/p/glossary/glossary_item/term
629,meaning,When a braked wheel loses adhesion with the ra...,,req_document/p/glossary/glossary_item/meaning
630,title,Other technical functions,11,req_document/p/title


In [6]:
# Create 'text_clean' attribute in df (list of tokens)
from utils import clean_data

df['text_clean'] = df['text'].apply(lambda x: clean_data.data_preparation(x))

df.head()

,tag,text,id,path,text_clean
0,title,ERTMS/ETCS Functional Requirements Specificati...,,req_document/title,"[ertm, etc, function, requir, specif, fr]"
1,version,5.00,,req_document/version,"[5, 00]"
2,issue_date,2007-06-21,,req_document/issue_date,"[2007, 06, 21]"
3,file_number,ERA/ERTMS/003204,,req_document/file_number,"[era, ertm, 003204]"
4,change_date,2007-06-21,,req_document/change_log/change_log_item/change...,"[2007, 06, 21]"


In [7]:
# Create a corpus
# clean_text = clean_data.df_tokenize(df['text_clean'], 2)
clean_text = df['text_clean']


In [9]:
 # Create a Dictionary: Associate each word in the corpus with a unique integer ID
from gensim import corpora

doc_dictionary = corpora.Dictionary(clean_text)



# Perplexity

```python
log_perplexity(chunk, total_docs=None)
```

Calculate and return per-word likelihood bound, using a chunk of documents as evaluation corpus.

Also output the calculated statistics, including the perplexity=2^(-bound), to log at INFO level.

Parameters
- chunk (list of list of (int, float)) – The corpus chunk on which the inference step will be performed.
- total_docs (int, optional) – Number of docs used for evaluation of the perplexity.

Returns
The variational bound score calculated for each word.

Return type
numpy.ndarray

In [18]:
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary

BOW = [doc_dictionary.doc2bow(text) for text in clean_text]

# Assuming you have a preprocessed corpus and dictionary
lda_model = LdaModel(BOW, num_topics=10, id2word=doc_dictionary)


# Calculate perplexity on the same corpus (not ideal, but for illustration)
perplexity = lda_model.log_perplexity(BOW, df.size)
print(f'Perplexity: {perplexity}')

Perplexity: -5.388787025174806
